# GANomaly 3D Notebook Evaluation

## Initial Configurations

### Model Hiperparameters tuning

In [1]:
# %load '../../models/ganomaly_3D/hiperparameters.py'
"""This file contains all the hiperparameters for GANomaly 3D model. You can modify this file in order to change default values stablished here.
Version: 1.5
Made by: Edgar Rangel
"""

def get_options():
    """This function return a dictionary with the hiperparameters and options to execute the GANomaly 3D model in any of the selected modes. It doesn't require any parameter."""

    opts = dict(
        gpus = '1', # ID of the GPU which will be used
        n_cpus = 16, # Number of CPU cores to use while running
        lr = 0.0002, # Learning rate
        dataset_path = "../../datasets/gait_v2/gait_v2.tfrecord", # Absolute path where the tfrecord is located to be used
        normal_class = 1, # Class label that will be the normal data in the training process
        kfolds = 5, # Number of kfolds in which the model will be evaluated with the tfrecord
        batch_size = 16, # Input batch size
        epochs = 20000, # Quantity of epochs to do in training
        seed = 8128, # Seed used to enable the replicability of the experiment
        save_path = "../../results/Ganomaly_3D/0004_eval_of_0002", # Path where the experiments will be saved
        save_frecuency = 1000, # Integer indicating between how many epochs the results and models will be saved
        gen_model_path = "../../results/Ganomaly_3D/0002_train_parkinson", # Path where the generator model (h5) is allocated and will be loaded to run trained models
        disc_model_path = "../../results/Ganomaly_3D/0002_train_parkinson", # Path where the discriminator model (h5) is allocated and will be loaded to run trained models
        eval_train = True, # If its True, then the loaded model will evaluate train data and test data together.
        isize = 64, # Input size of the videos, e.g. 64 equals to videos with shape 64x64x64
        nc = 1, # Quantity of channels in the data
        nz = 100, # Context vector size
        ngf = 64, # Quantity of initial filters in the first convolution of the encoder
        extra_layers = 0, # Quantity of layer blocks to add before reduction
        w_adv = 1, # Adversarial loss weight
        w_con = 50, # Contextual loss weight
        w_enc = 1, # Encoder loss weight
        beta_1 = 0.5, # Momentum of beta 1 in adam optimizer for generator and discriminator
        beta_2 = 0.999, # Momentum of beta 2 in adam optimizer for generator and discriminator
        readme = """This file contains information about the experiment made in this instance.

All models saved don't include the optimizer, but this file explains how to train in the same conditions.

Basic notation:

- {i}_Ganomaly3D-{size}x{size}x{size}x{nc}: Experiment id, name of the model and input dimension of model.
- H x W x F, F x H x W x C or H x W x C: Data dimensions used where F are frames, H height, W width and C channels.

Experiment settings:
- The seed used was {seed} for python random module, numpy random and tf random after the library importations.
- The batch size was of {batch}.
- The optimizer used in this experiment was Adam for generator and discriminator.
- The number of classes in this dataset are 2 (Normal and Parkinson) .
- This experiment use the data of gait_v2/dataset_09-jun-2022 tfrecord.
- The initial lr was of {lr}.
- The beta 1 and beta 2 for adam optimizer was {beta_1} and {beta_2} respectively.
- The total epochs made in this experiment was of {epochs}.
- The context vector size (nz) was of {nz}.
- The # channels in data (nc) was of {nc}.
- The initial filters in the first convolution of the encoder was {ngf}.
- The quantity of layer blocks to add before reduction was of {extra_layers}.
- The weights for adversarial, contextual and encoder error respectively in generator were {w_gen}.

Transformations applied to data (following this order):
- Resize: We resize the frames of volumes to H x W ({size} x {size}).
- Equidistant Oversampling volume: We take {size} frames sampled equidistant of volumes to train and test the data.
- Convert: We convert the videos in RGB to Grayscale.
- Normalize: We normalize the volume with mean and std of 0.5 for both.
- Scale: We scale the data between -1 and 1 using min max scaler to be comparable with generated images.
- Identify: We identify each video per patient with an integer value.
- Randomize: We randomize the order of samples in every epoch.

Training process:
- The data doesn't have train and test partition but we make the partitions like this:
    * ~80% (11 patients) of normal (parkinson) data is used in train for kfold {k}.
    * ~20% (3 patients) of normal (parkinson) data is used in test for kfold {k}.
    * ~20% (3 patients) of abnormal (healthy) data are used in test for kfold {k}.
"""
    )

    opts["w_gen"] = (opts["w_adv"], opts["w_con"], opts["w_enc"])
    
    return opts

opts = get_options()

### Selecting the device to work with

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = opts["gpus"]

### Libraries import

In [3]:
import sys
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold

sys.path.append("../../")

2022-08-03 01:41:51.329039: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Model functions import

In [4]:
from datasets.gait_v2.extraction_methods import get_data
from models.ganomaly_3D.modes.eval import exec_loop as test
from models.ganomaly_3D.data_preprocessing import preprocess_gait_dataset
from utils.metrics import get_true_positives, get_true_negatives, get_false_positives, get_false_negatives, get_AUC, get_mean

### GPU Memory Configuration

In [5]:
if os.getenv("CUDA_VISIBLE_DEVICES") != '-1':
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
tf.debugging.set_log_device_placement(False)

## Dataset pre processing

In [6]:
total_data = get_data(opts["dataset_path"], opts["n_cpus"])
total_data

2022-08-03 01:42:00.207678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-03 01:42:02.699575: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7374 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:1c:00.0, compute capability: 8.6


<ParallelMapDataset element_spec=(TensorSpec(shape=(None, None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [7]:
shape_videos = []
labels_videos = []
patients_ids = []
for x, y, z in total_data:
    shape_videos.append(x.numpy().shape)
    labels_videos.append(y.numpy())
    patients_ids.append(z.numpy())
shape_videos = np.r_[shape_videos]
labels_videos = np.r_[labels_videos]
patients_ids = np.r_[patients_ids]
print("Data information about the data")
print("Total videos: ", shape_videos.shape[0])
print("Min value of frames: ", np.min(shape_videos[:,0]))
print("Max value of frames: ", np.max(shape_videos[:,0]))
print("Mean value of frames: ", np.mean(shape_videos[:,0]))
print("Unique ids: ", np.unique(patients_ids))

Data information about the data
Total videos:  240
Min value of frames:  72
Max value of frames:  387
Mean value of frames:  144.6375
Unique ids:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


In [8]:
ns = {i:0 for i in np.unique(labels_videos)}
videos_4_pat = {i:0 for i in np.unique(patients_ids)}
for i, forma in enumerate(shape_videos):
    frames = opts["isize"]
    to_sum = np.ceil(forma[0] / frames).astype(np.int64)
    videos_4_pat[patients_ids[i]] += to_sum
    ns[labels_videos[i]] += to_sum
for i in ns:
    print("Video clips for label {}: {}".format(i, ns[i]))

Video clips for label 0: 351
Video clips for label 1: 313


In [9]:
normal_patients_ids = np.unique(patients_ids[labels_videos == opts['normal_class']])
abnormal_patients_ids = np.unique(patients_ids[labels_videos != opts['normal_class']])

normal_patients_ids, abnormal_patients_ids

(array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 26, 28, 29]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 23, 24, 25, 27, 30]))

In [10]:
normal_patients, abnormal_patients = preprocess_gait_dataset(
    total_data, 
    opts,
    normal_patients_ids,
    abnormal_patients_ids
)
normal_patients, abnormal_patients

([<CacheDataset element_spec=(TensorSpec(shape=(64, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  <CacheDataset element_spec=(TensorSpec(shape=(64, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  <CacheDataset element_spec=(TensorSpec(shape=(64, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  <CacheDataset element_spec=(TensorSpec(shape=(64, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  <CacheDataset element_spec

## Model pre requisites

### Data partitions

In [11]:
kfolds = opts["kfolds"]
seed = opts["seed"]
kf = KFold(n_splits=kfolds, shuffle=True, random_state=seed)
train_folds = []
test_folds = []
test_totals = [0] * kfolds
for k, (train_indexes, test_indexes) in enumerate(kf.split(normal_patients)):
    data = normal_patients[train_indexes[0]]
    total_samples = videos_4_pat[normal_patients_ids[train_indexes[0]]]
    for i in range(1, len(train_indexes)):
        data = data.concatenate(normal_patients[train_indexes[i]])
        total_samples += videos_4_pat[normal_patients_ids[train_indexes[i]]]
    train_folds.append(
        data.shuffle(
            total_samples, 
            reshuffle_each_iteration=True
        ).batch(
            opts['batch_size']
        ).prefetch(-1)
    )

    data = normal_patients[test_indexes[0]]
    test_totals[k] += videos_4_pat[normal_patients_ids[test_indexes[0]]]
    for i in range(1, len(test_indexes)):
        data = data.concatenate(normal_patients[test_indexes[i]])
        test_totals[k] += videos_4_pat[normal_patients_ids[test_indexes[i]]]
    test_folds.append(data)

    print("Kfold {}\n\tNormal train ids: {}\n\tNormal test ids: {}".format(
        k + 1,
        [normal_patients_ids[i] for i in train_indexes],
        [normal_patients_ids[i] for i in test_indexes]
    ))

for k , (_, test_indexes) in enumerate(kf.split(abnormal_patients)):
    data = abnormal_patients[test_indexes[0]]
    test_totals[k] += videos_4_pat[abnormal_patients_ids[test_indexes[0]]]
    for i in range(1, len(test_indexes)):
        data = data.concatenate(abnormal_patients[test_indexes[i]])
        test_totals[k] += videos_4_pat[abnormal_patients_ids[test_indexes[i]]]
    test_folds[k] = test_folds[k].concatenate(
        data
    ).shuffle(
        test_totals[k], 
        reshuffle_each_iteration=True
    ).batch(
        opts['batch_size']
    ).prefetch(-1)

    print("Kfold {}\n\tAbnormal test ids: {}".format(
        k + 1,
        [abnormal_patients_ids[i] for i in test_indexes]
    ))

Kfold 1
	Normal train ids: [12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 28]
	Normal test ids: [19, 26, 29]
Kfold 2
	Normal train ids: [12, 15, 17, 18, 19, 20, 21, 22, 26, 28, 29]
	Normal test ids: [13, 14, 16]
Kfold 3
	Normal train ids: [12, 13, 14, 16, 17, 18, 19, 21, 26, 28, 29]
	Normal test ids: [15, 20, 22]
Kfold 4
	Normal train ids: [13, 14, 15, 16, 19, 20, 21, 22, 26, 28, 29]
	Normal test ids: [12, 17, 18]
Kfold 5
	Normal train ids: [12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 26, 29]
	Normal test ids: [21, 28]
Kfold 1
	Abnormal test ids: [8, 9, 25, 30]
Kfold 2
	Abnormal test ids: [3, 4, 5]
Kfold 3
	Abnormal test ids: [2, 11, 27]
Kfold 4
	Abnormal test ids: [6, 7, 23]
Kfold 5
	Abnormal test ids: [1, 10, 24]


### Metrics creation

In [13]:
TP = get_true_positives()
TN = get_true_negatives()
FP = get_false_positives()
FN = get_false_negatives()
AUC = get_AUC()

### Loop execution

In [ ]:
load_path = '../../results/Ganomaly_3D/0002_train_parkinson/'
for k, folder in enumerate(sorted(os.listdir(load_path))):
    opts["gen_model_path"] = os.path.join(load_path, folder, "gen_model.h5")
    opts["disc_model_path"] = os.path.join(load_path, folder, "disc_model.h5")
    
    test(
        opts, 
        TP,
        TN,
        FP,
        FN,
        AUC,
        train_folds[k],
        test_folds[k]
    )

2022-08-03 02:11:58.381075: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1 of 243
2022-08-03 02:11:58.381174: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 2 of 243
2022-08-03 02:11:58.381280: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 3 of 243
2022-08-03 02:13:58.916085: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 25 of 243
2022-08-03 02:13:58.916190: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 26 of 243
2022-08-03 02:13:58.916201: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 27 of 243
2022-08-03 02:13:59.530575: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may

2022-08-03 02:22:23.947491: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 126 of 243
2022-08-03 02:22:23.947650: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 127 of 243
2022-08-03 02:22:23.947711: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 128 of 243
2022-08-03 02:22:24.579407: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 129 of 243
2022-08-03 02:22:24.579464: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 130 of 243
2022-08-03 02:22:24.579476: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 131 of 243
2022-08-03 02:22:24.867569: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer 